In [1]:
import pandas as pd
from main import result_path
import numpy as np
import os
%load_ext autoreload
%autoreload 2

In [132]:
dataset_name = 'Compas'
# path = result_path(dataset_name)
methods = os.listdir(f'results/{dataset_name}')
methods = list(filter(lambda x: "ImprovedModel" in x, methods))

In [133]:

# methond_names = ['baseline', 'ARL', 'ImprovedModel']

methond_names = ['baseline', 'ARL']

if dataset_name == 'Adult':
    methond_names+=['ImprovedModel_lr:0.001_corr:0.1_adv_mul:0.01_switch:1']
elif dataset_name == 'Compas':
    methond_names+=['ImprovedModel_lr:0.01_corr:0.05_adv_mul:0.05_switch:10']
    

# methond_names += methods

# for corr in [0.1, 0.2, 0.5, 0.05]:
#     for disc in [0.1, 0.2, 0.5, 0.05, 1]:
#         methond_names += [f"ImprovedModel_corr_{corr}_{disc}"]
        
        
df = pd.DataFrame()
for method_name in methond_names:
    curr_df = pd.DataFrame()
    try:
        for seed in range(20):
            path = result_path(dataset_name, method_name, seed)
            seed_df = pd.read_csv(path, index_col='Unnamed: 0')
            curr_df = pd.concat([curr_df, seed_df])
            
    except Exception as e:
#         print(f"read only {len(curr_df)} seeds.")
        pass

    curr_df = curr_df.mean()
    curr_df.name = (f"{dataset_name}_{method_name}")
    df = pd.concat([df, curr_df], axis=1)
    
df = df.T
features_to_display = ['Accuracy', 'min subgroup AUC', 'average subgroup AUC', 'minority subgroup AUC']
DeltaAccuracy_features = list(filter(lambda x: ' group' in x and 'DAccuracy' in x, df.columns))
# features_to_display += DeltaAccuracy_features
df['avg. DAccuracy'] = df[DeltaAccuracy_features].mean(axis=1)

features_to_display += ['corr', 'Test WSC', 'avg. DAccuracy', 'HSIC', 'Test Delta WSC']
df['HSIC'] = np.log10(df['HSIC'])
df = df.sort_values(by=['average subgroup AUC'], ascending=False)
df = df.rename(index={list(filter(lambda idx: 'Improved' in idx, df.index))[0]: dataset_name+'_AOL'})
df = df[features_to_display].round(3)
df = df[['Accuracy', 'min subgroup AUC', 'average subgroup AUC', 'minority subgroup AUC']]

In [134]:
df

,Accuracy,min subgroup AUC,average subgroup AUC,minority subgroup AUC
Compas_AOL,0.674,0.648,0.718,0.764
Compas_baseline,0.676,0.646,0.717,0.756
Compas_ARL,0.677,0.648,0.717,0.751


In [135]:
print(df.round(3).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  Accuracy &  min subgroup AUC &  average subgroup AUC &  minority subgroup AUC \\
\midrule
Compas\_AOL      &     0.674 &             0.648 &                 0.718 &                  0.764 \\
Compas\_baseline &     0.676 &             0.646 &                 0.717 &                  0.756 \\
Compas\_ARL      &     0.677 &             0.648 &                 0.717 &                  0.751 \\
\bottomrule
\end{tabular}



SyntaxError: invalid syntax (<ipython-input-117-8bca0aef6741>, line 1)

In [118]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'font-weight: bold'
    #remove % and cast to float
    data = data.replace('%','', regex=True).astype(float)
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

In [120]:
df = df.style.apply(highlight_max)

In [121]:
df.style.format('${0:,.3f}')

AttributeError: 'Styler' object has no attribute 'style'

In [49]:

# methond_names = ['baseline', 'ARL', 'ImprovedModel']

# methond_names = ['baseline', 'ARL']
methond_names = methods

# for corr in [0.1, 0.2, 0.5, 0.05]:
#     for disc in [0.1, 0.2, 0.5, 0.05, 1]:
#         methond_names += [f"ImprovedModel_corr_{corr}_{disc}"]
        
        
df = pd.DataFrame()
for method_name in methond_names:
    curr_df = pd.DataFrame()
    try:
        for seed in range(20):
            
            path = result_path(dataset_name, method_name, seed)
            seed_df = pd.read_csv(path, index_col='Unnamed: 0')
            curr_df = pd.concat([curr_df, seed_df])
            
    except Exception as e:
        print(f"read only {len(curr_df)} seeds.")
        pass

    curr_df = curr_df.mean()
    curr_df.name = (f"{dataset_name}_{method_name}")
    df = pd.concat([df, curr_df], axis=1)
    
df = df.T

features_to_display = ['Accuracy', 'min subgroup AUC', 'average subgroup AUC', 'minority subgroup AUC']
DeltaAccuracy_features = list(filter(lambda x: ' group' in x and 'DAccuracy' in x, df.columns))
features_to_display += DeltaAccuracy_features
df['avg. DAccuracy'] = df[DeltaAccuracy_features].mean(axis=1)


features_to_display += ['corr', 'Test WSC', 'avg. DAccuracy', 'HSIC', 'Test Delta WSC']
df['HSIC'] = np.log10(df['HSIC'])
df = df.sort_values(by=['min subgroup AUC'], ascending=False)

df[features_to_display]#.head()


,Accuracy,min subgroup AUC,average subgroup AUC,minority subgroup AUC,DAccuracy sex group 0,DAccuracy sex group 1,DAccuracy race group 0,DAccuracy race group 1,DAccuracy race group 2,corr,Test WSC,avg. DAccuracy,HSIC,Test Delta WSC
Compas_ImprovedModel_lr:0.01_corr:0.05_adv_mul:0.05_switch:10,0.674180,0.648377,0.717932,0.764269,0.020687,0.005095,0.008154,0.023254,0.015445,0.032272,0.634336,0.014527,-3.547835,0.051317
Compas_ImprovedModel_lr:0.01_corr:0.05_adv_mul:0.1_switch:10,0.674134,0.647391,0.717724,0.767122,0.021997,0.005422,0.007770,0.022883,0.014955,0.032177,0.641263,0.014605,-3.545051,0.044801
Compas_ImprovedModel_lr:0.01_corr:0.05_adv_mul:0.01_switch:10,0.673533,0.647143,0.717653,0.758739,0.022362,0.005480,0.008275,0.025131,0.016056,0.031688,0.616818,0.015461,-3.548731,0.058294
Compas_ImprovedModel_lr:0.01_corr:0.1_adv_mul:0.01_switch:10,0.674965,0.644501,0.716784,0.751725,0.018702,0.004567,0.009283,0.022709,0.016628,0.032169,0.605344,0.014378,-3.557143,0.075096
Compas_ImprovedModel_lr:0.01_corr:0.5_adv_mul:0.01_switch:10,0.678545,0.648508,0.716654,0.761245,0.018186,0.004477,0.008217,0.027343,0.016789,0.031682,0.629498,0.015002,-3.554589,0.060474
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Compas_ImprovedModel_lr:0.0001_corr:0.5_adv_mul:0.05_switch:5,0.609885,0.587500,0.633170,0.649234,0.016463,0.003957,0.009194,0.021806,0.014313,0.035875,0.523913,0.013146,-3.278441,0.086186
Compas_ImprovedModel_lr:0.0001_corr:0.5_adv_mul:0.05_switch:10,0.607483,0.574423,0.630961,0.647066,0.018136,0.004366,0.009102,0.016332,0.013748,0.034716,0.494872,0.012337,-3.300102,0.112611
Compas_ImprovedModel_lr:0.0001_corr:0.5_adv_mul:0.1_switch:5,0.604065,0.572772,0.623911,0.635778,0.016428,0.003935,0.008421,0.021046,0.012569,0.033510,0.524402,0.012480,-3.279614,0.079876
Compas_ImprovedModel_lr:0.0001_corr:0.5_adv_mul:0.1_switch:1,0.601363,0.571062,0.622049,0.637040,0.017421,0.004236,0.008360,0.019593,0.010464,0.033807,0.497635,0.012015,-3.281374,0.103727
